In [1]:
import json
import ast

import pandas as pd
from datetime import datetime, timedelta
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind
from tqdm.auto import tqdm
from scipy.stats import levene
import scipy.stats as st
import pylab
import statsmodels.stats.api as sms
import phik

import my_hypotheses as hyp
import LichessAnalys as li
import my_message as ms

c:\Program Files\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
lichessAnalys = li.LichessAnalys()
hypotheses = hyp.ProgressivePlayerCanBeACheater()
message_to_send = ms.MessageToSend()

In [78]:
df_for_test_group = pd.read_csv('./move_score_and_clocks.csv', sep=',')

In [37]:
df_for_control_group = \
     pd.read_csv('./merge_eval_and_clocks_for_control_group.csv', sep=',')

In [62]:
df_for_test_group

,date,game_id,time_control,clocks_list,move_count,user_id,move_score
0,2023-05-09,2vKAIGqk,45+45,"[12.36, 2.44, 1.64, 19.56, 4.84, 89.92, 164.48...",43,dhmayer,"[36, 0, 0, 0, 0, 8, 0, 28, 0, 15, 22, 89, 91, ..."
1,2023-04-25,kYVEqjDZ,45+15,"[6.92, 6.84, 5.16, 8.44, 8.4, 12.92, 9.52, 10....",28,dhmayer,"[36, 29, 34, 31, 29, 46, 36, 33, 46, 39, 46, 2..."
2,2023-04-18,u9snz9M7,25+2,"[1.76, 0.16, 3.6, 4.72, 6.8, 0.32, 31.2, 11.36...",46,dhmayer,"[36, 30, 23, 12, 13, 0, 0, 18, 20, -54, 57, 54..."
3,2023-04-14,VkXvtojd,15+15,"[2.2, 2.28, 5.88, 6.12, 1.88, 2.36, 2.92, 3.64...",40,dhmayer,"[36, 30, 17, 39, 44, 38, 13, 85, 135, 117, 150..."
4,2023-04-12,ZXKWe43X,15+15,"[4.2, 3.72, 4.68, 5.08, 5.08, 6.68, 4.56, 5.72...",51,dhmayer,"[36, 40, 44, 52, 64, 72, 24, -49, 44, 45, 52, ..."
5,2023-04-12,1wgDNg2a,15+15,"[1.4, 3.24, 2.76, 1.48, 3.8, 1.96, 12.92, 4.2,...",50,dhmayer,"[36, 40, 44, 52, 64, 72, 67, 66, -308, -478, -..."
6,2023-04-12,Cara9VC5,15+15,"[1.16, 0.76, 1.0, 1.08, 0.04, 1.32, 1.64, 7.24...",53,dhmayer,"[36, 30, 18, 36, 28, 56, 0, 64, 60, 55, 52, 63..."
7,2023-04-11,FoF5X0yC,15+15,"[5.64, 3.0, 1.72, 1.72, 7.0, 4.36, 9.84, 10.28...",32,dhmayer,"[0, 16, 75, 80, 13, 76, 50, 54, 63, 50, 58, 34..."
8,2023-04-11,tHftXu0B,15+15,"[1.48, 1.88, 1.32, 1.08, 1.0, 1.24, 1.16, 1.16...",41,dhmayer,"[36, 29, 34, 31, 29, 46, 36, 33, 50, 36, 42, 3..."
9,2023-04-10,SEWooq3k,15+15,"[2.2, 1.8, 7.72, 7.08, 2.44, 2.04, 2.92, 6.0, ...",46,dhmayer,"[36, 28, 0, -22, -1, 0, -7, -17, -17, 0, -22, ..."


In [70]:
    df_for_test_group['clocks_list_new'] = \
    [ast.literal_eval(i) for i in df_for_test_group['clocks_list']]
    df_for_test_group['move_score_new'] = \
    [ast.literal_eval(i) for i in df_for_test_group['move_score']]

    len_df = df_for_test_group.shape[0]
    list_df_len = list(range(0, len_df, 1))
    correlation_list = []

    for i in list_df_len:
        len_clocks = len(df_for_test_group['clocks_list_new'][i])
        print(i)
        print(len_clocks)
        len_score = len(df_for_test_group['move_score_new'][i])
        print(len_score)
        if len_clocks != len_score:
            if len_clocks > len_score:
                print(len_clocks, len_score)
                df_for_test_group['clocks_list_new'][i].pop(-1)
                print(len(df_for_test_group['clocks_list_new'][i]), len(df_for_test_group['move_score_new'][i]))
            else:
                print(len_clocks, len_score)
                df_for_test_group['move_score_new'][i].pop(-1)
                print(len(df_for_test_group['clocks_list_new'][i]), len(df_for_test_group['move_score_new'][i]))

0
43
43
1
28
28
2
47
46
47 46
46 46
3
41
40
41 40
40 40
4
51
51
5
50
50
6
53
53
7
32
31
32 31
31 31
8
41
41
9
46
46
10
86
86
11
47
46
47 46
46 46
12
31
31


In [77]:
df_for_test_group['clocks_list_new'] = \
[ast.literal_eval(i) for i in df_for_test_group['clocks_list']]
df_for_test_group['move_score_new'] = \
[ast.literal_eval(i) for i in df_for_test_group['move_score']]

len_df = df_for_test_group.shape[0]
list_df_len = list(range(0, len_df, 1))
correlation_list = []

for i in list_df_len:
    len_clocks = len(df_for_test_group['clocks_list_new'][i])
    len_score = len(df_for_test_group['move_score_new'][i])
    if len_clocks != len_score: 
        if len_clocks > len_score:
            df_for_test_group['clocks_list_new'][i].pop(-1)
        else:
            df_for_test_group['move_score_new'][i].pop(-1)


43 43
28 28
46 46
40 40
51 51
50 50
53 53
31 31
41 41
46 46
86 86
46 46
31 31


In [59]:
l = [0, 10, 20, 30, 40, 50]

popped_item = l.pop(-1)
print(l)

[0, 10, 20, 30, 40]


In [52]:
len(df_for_test_group['clocks_list_new'][2][:-1])

46

In [ ]:
len(df_for_test_group['move_score_new'][2])

46

In [ ]:
l = df_for_test_group['clocks_list_new'][0]

In [79]:
def add_correlation_coefficient(
    df_for_satatistical_test: pd.DataFrame) -> pd.Series:
    df_for_satatistical_test['clocks_list_new'] = \
    [ast.literal_eval(i) for i in df_for_satatistical_test['clocks_list']]
    df_for_satatistical_test['move_score_new'] = \
    [ast.literal_eval(i) for i in df_for_satatistical_test['move_score']]

    len_df = df_for_satatistical_test.shape[0]
    list_df_len = list(range(0, len_df, 1))
    correlation_list = []

    for i in list_df_len:
        len_clocks = len(df_for_satatistical_test['clocks_list_new'][i])
        len_score = len(df_for_satatistical_test['move_score_new'][i])
        if len_clocks != len_score: 
            if len_clocks > len_score:
                df_for_satatistical_test['clocks_list_new'][i].pop(-1)
            else:
                df_for_satatistical_test['move_score_new'][i].pop(-1)
            d = {
            'clocks_list': df_for_satatistical_test['clocks_list_new'][i],
            'move_score': df_for_satatistical_test['move_score_new'][i]
            }
            df = pd.DataFrame(data=d)
            t = df[['clocks_list', 'move_score']]
            phik_overview = t.phik_matrix()
            correlation_coefficient = phik_overview['clocks_list'][1]
            correlation_list.append(correlation_coefficient)
        else:
            d = {
            'clocks_list': df_for_satatistical_test['clocks_list_new'][i],
            'move_score': df_for_satatistical_test['move_score_new'][i]
            }
            df = pd.DataFrame(data=d)
            t = df[['clocks_list', 'move_score']]
            phik_overview = t.phik_matrix()
            correlation_coefficient = phik_overview['clocks_list'][1]
            correlation_list.append(correlation_coefficient)
    result = pd.Series(correlation_list)
    return result

In [80]:
add_correlation_coefficient = add_correlation_coefficient(
        df_for_satatistical_test=df_for_test_group)
add_correlation_coefficient \
    .to_csv('./add_correlation_coefficient.csv',
    index=False,
    sep=",")

interval columns not set, guessing: ['clocks_list', 'move_score']
interval columns not set, guessing: ['clocks_list', 'move_score']
interval columns not set, guessing: ['clocks_list', 'move_score']
interval columns not set, guessing: ['clocks_list', 'move_score']
interval columns not set, guessing: ['clocks_list', 'move_score']
interval columns not set, guessing: ['clocks_list', 'move_score']
interval columns not set, guessing: ['clocks_list', 'move_score']
interval columns not set, guessing: ['clocks_list', 'move_score']
interval columns not set, guessing: ['clocks_list', 'move_score']
interval columns not set, guessing: ['clocks_list', 'move_score']
interval columns not set, guessing: ['clocks_list', 'move_score']
interval columns not set, guessing: ['clocks_list', 'move_score']
interval columns not set, guessing: ['clocks_list', 'move_score']


In [81]:
print(add_correlation_coefficient)

0     0.000000
1     0.200291
2     0.528902
3     0.472703
4     0.698665
5     0.526515
6     0.000000
7     0.000000
8     0.270530
9     0.000000
10    0.288534
11    0.000000
12    0.000000
dtype: float64


In [82]:
def levene_test(df_for_control_group: pd.DataFrame,
    df_for_test_group: pd.DataFrame) -> pd.DataFrame:
    result = pd.DataFrame(columns=[
            'levene_p_value_list'])

    df_for_test_group['clocks_list_new'] = \
    [ast.literal_eval(i) for i in df_for_test_group['clocks_list']]
    df_for_control_group['clocks_list_new'] = \
    [ast.literal_eval(i) for i in df_for_control_group['clocks_list']]

    test_group = df_for_test_group['clocks_list_new']
    control_group = df_for_control_group['clocks_list_new']

    for i in test_group:
        levene_p_value_list = []
        for k in control_group:
            stat, p = levene(k, i, center='mean')
            levene_p_value_list.append(round(p, 3))
        columns = {'levene_p_value_list': str(levene_p_value_list)}
        df = pd.DataFrame(data=columns, index=[0])
        result = \
        pd.concat([result, \
            df], ignore_index=True , join="outer")

    result['levene_p_value_list'] = \
    [ast.literal_eval(i) for i in result['levene_p_value_list']]

    leven_median = []
    for i in result['levene_p_value_list']:
        leven_median.append(np.median(i))
    result['levene_p_value_median'] = leven_median
    result['game_id'] = df_for_test_group['game_id']
    result = result.drop(columns=['levene_p_value_list'])
    return result

In [83]:
levene_test = levene_test(
    df_for_control_group=df_for_control_group,
    df_for_test_group=df_for_test_group)
levene_test \
    .to_csv('./levene_test.csv',
    index=False,
    sep=",")

In [84]:
lev = pd.read_csv('./levene_test.csv',
    sep=",")
lev


,levene_p_value_median,game_id
0,0.000,2vKAIGqk
1,0.019,kYVEqjDZ
2,0.003,u9snz9M7
3,0.035,VkXvtojd
4,0.038,ZXKWe43X
5,0.068,1wgDNg2a
6,0.019,Cara9VC5
7,0.029,FoF5X0yC
8,0.080,tHftXu0B
9,0.008,SEWooq3k


In [87]:
user_id = 'dhmayer'

In [88]:
def combining_main_and_statistical_data(
    user_id: str,
    levene_test: pd.DataFrame,
    merge_eval_and_clocks: pd.DataFrame,
    add_correlation_coefficient: pd.Series) -> pd.DataFrame:
    user_id = user_id
    df = merge_eval_and_clocks.query('user_id == @user_id')
    levene_test['correlation_coefficient']= add_correlation_coefficient
    result = df.merge(levene_test,
            on='game_id',
            how='left')
    return result

In [89]:
df_classical = pd.read_csv('./df_classical.csv', sep=',')

In [90]:
combining_main_and_statistical_data = combining_main_and_statistical_data(
    user_id=user_id,
    levene_test=levene_test,
    merge_eval_and_clocks=df_classical,
    add_correlation_coefficient=add_correlation_coefficient)
combining_main_and_statistical_data \
    .to_csv('./combining_main_and_statistical_data.csv',
    index=False,
    sep=",")

In [91]:
combining_main_and_statistical_data

,game_id,date,game_speed,rating,rating_diff,clocks_mean,clocks_std,clocks_median,move_count,time_control,user_id,mistake,blunder,inaccuracy,acpl,levene_p_value_median,correlation_coefficient
0,2vKAIGqk,2023-05-09,classical,2224,14,76.07,114.09,32.0,43,45+45,dhmayer,2,2,6,40,0.000,0.000000
1,kYVEqjDZ,2023-04-25,classical,2217,7,35.32,109.58,7.5,28,45+15,dhmayer,1,0,0,11,0.019,0.200291
2,u9snz9M7,2023-04-18,classical,2215,2,28.62,42.75,5.0,46,25+2,dhmayer,0,4,7,42,0.003,0.528902
3,VkXvtojd,2023-04-14,classical,2204,11,11.98,21.10,4.0,40,15+15,dhmayer,1,1,5,49,0.035,0.472703
4,QMgw401c,2023-04-13,classical,2187,17,21.00,29.21,5.5,18,15+15,dhmayer,0,1,0,34,NaN,NaN
5,ZXKWe43X,2023-04-12,classical,2180,7,21.33,28.84,9.0,51,15+15,dhmayer,1,0,7,21,0.038,0.698665
6,1wgDNg2a,2023-04-12,classical,2172,8,23.08,38.89,7.5,50,15+15,dhmayer,1,1,2,42,0.068,0.526515
7,Cara9VC5,2023-04-12,classical,2162,10,8.79,18.33,3.0,53,15+15,dhmayer,1,2,5,38,0.019,0.000000
8,FoF5X0yC,2023-04-11,classical,2149,13,15.12,20.33,6.5,32,15+15,dhmayer,1,0,7,34,0.029,0.000000
9,tHftXu0B,2023-04-11,classical,2128,21,18.54,32.94,3.0,41,15+15,dhmayer,1,0,2,17,0.080,0.270530
